# Need to figure out how to confirm that the format is correct...

In [ ]:
# FORMAT TESTING....

# Upload

In [2]:
import pandas as pd
import sqlite3
from helper_functions import upload_new_race_results

In [ ]:
# Set the database path to a location with write permissions
db_path = '../race_league_results.db'

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


In [ ]:
path = 'data/2024/240218FnGResultsBoth.csv'
race_date = '02/18/2024'
race_id = 240218
description = '2024 F&G Race #4'
N_tiers=13
N_teams=8 

""" # Comment out for now
upload_new_race_results(
    path,
    race_date,
    race_id,
    description,
    N_tiers,
    N_teams,
    conn
)
"""